In [1]:
# import torch

# torch.set_default_device('cuda')
# torch.set_default_dtype(torch.float32)

# mod = torch.nn.Linear(20, 30)
# print(mod.weight.device)
# print(mod(torch.randn(128, 20)).device)

In [2]:
# # Set default tensor type to CUDA if available
# if torch.cuda.is_available():
#     torch.set_default_tensor_type(torch.cuda.FloatTensor)
# else:
#     print("CUDA is not available. Using CPU instead.")


In [3]:
import os

os.environ['TRANSFORMERS_CACHE'] = '/workspace/transformers_cache/'

In [4]:
!ls ../data

first_2k_fixed.csv  linkedin-2023-q4-sm.csv.gz	naics_descriptions.csv


In [5]:
import pandas as pd

df = pd.read_csv("../data/first_2k_fixed.csv")

In [6]:
df.head()

,Commercial Name,Business Tags,Short Description,Long Description,Main Business Category,predicted_naics,pred_probabilities
0,White Horse,Tile Manufacturing | European Aesthetics Ceram...,White Horse is highly regarded as a tile trail...,White Horse Ceramic Singapore is a leading man...,Tile Store,327,99.999980
1,Wealth Solution Partners,Super and SMSF Services | Financial Planning a...,"WSP, Wealth Solution Partners, Financial Plann...",Wealth Solution Partners Pty Ltd is an indepen...,Investment Consultants & Financial Advisors,523,99.999995
2,PMG,Fire and Water Cleanup Services | Mold Remedia...,PMG General Solutions Inc. is an environmental...,PMG General Solutions Inc. is an environmental...,Damage Restoration & Mold Remediation,562,99.999975
3,TMP Capital PLLC,Licensed in AL & FL | 203K Loans | 15-year Fix...,TMP Capital PLLC Consulting Company Franklin M...,"TMP Capital PLLC Consulting Company, also know...",Mortgage Brokers,522,99.999578
4,Genertek Power,Industrial and Commercial Energy Storage | Ass...,Genertek Power Ltd a UK electricity systems & ...,Genertek Power Limited is a privately-owned UK...,Renewable energy companies,221,99.999999


In [7]:
df['X'] = df.apply(lambda x: ' '.join(x[:4]).strip(), axis=1)
df.head()

,Commercial Name,Business Tags,Short Description,Long Description,Main Business Category,predicted_naics,pred_probabilities,X
0,White Horse,Tile Manufacturing | European Aesthetics Ceram...,White Horse is highly regarded as a tile trail...,White Horse Ceramic Singapore is a leading man...,Tile Store,327,99.999980,White Horse Tile Manufacturing | European Aest...
1,Wealth Solution Partners,Super and SMSF Services | Financial Planning a...,"WSP, Wealth Solution Partners, Financial Plann...",Wealth Solution Partners Pty Ltd is an indepen...,Investment Consultants & Financial Advisors,523,99.999995,Wealth Solution Partners Super and SMSF Servic...
2,PMG,Fire and Water Cleanup Services | Mold Remedia...,PMG General Solutions Inc. is an environmental...,PMG General Solutions Inc. is an environmental...,Damage Restoration & Mold Remediation,562,99.999975,PMG Fire and Water Cleanup Services | Mold Rem...
3,TMP Capital PLLC,Licensed in AL & FL | 203K Loans | 15-year Fix...,TMP Capital PLLC Consulting Company Franklin M...,"TMP Capital PLLC Consulting Company, also know...",Mortgage Brokers,522,99.999578,TMP Capital PLLC Licensed in AL & FL | 203K Lo...
4,Genertek Power,Industrial and Commercial Energy Storage | Ass...,Genertek Power Ltd a UK electricity systems & ...,Genertek Power Limited is a privately-owned UK...,Renewable energy companies,221,99.999999,Genertek Power Industrial and Commercial Energ...


In [8]:
df.iloc[0]['X']

'White Horse Tile Manufacturing | European Aesthetics Ceramic Tiles | Ceramic Mosaics | Glossy Tile | Green-certified Tiles | Clearance Items | Ceramic Tiles Manufacturer | Quartz Surfaces White Horse is highly regarded as a tile trailblazer, designing ceramic tiles of the highest quality inspired by European aesthetics. White Horse Ceramic Singapore is a leading manufacturer of high-quality ceramics, offering a wide range of ceramic products for both residential and commercial spaces. Established in 1998, the company is known for its commitment to quality, innovation, and artistry. Their skilled artisans meticulously craft every piece of ceramic to create beautiful and functional products that stand the test of time. With a global presence in over 50 countries, White Horse Ceramics is synonymous with style, durability, and unmatched craftsmanship. They offer a variety of tiles, including ceramic wall, floor, and porcelain tiles, as well as stone, mosaic, and quartz tiles. The company 

In [9]:
df_all_data = df[['X', 'predicted_naics']]
df_all_data.columns = ['X', 'y']
df_all_data.head(1)['X']

0    White Horse Tile Manufacturing | European Aest...
Name: X, dtype: object

In [10]:
df_all_data.head(1)['X'].astype(str).iloc[0]

'White Horse Tile Manufacturing | European Aesthetics Ceramic Tiles | Ceramic Mosaics | Glossy Tile | Green-certified Tiles | Clearance Items | Ceramic Tiles Manufacturer | Quartz Surfaces White Horse is highly regarded as a tile trailblazer, designing ceramic tiles of the highest quality inspired by European aesthetics. White Horse Ceramic Singapore is a leading manufacturer of high-quality ceramics, offering a wide range of ceramic products for both residential and commercial spaces. Established in 1998, the company is known for its commitment to quality, innovation, and artistry. Their skilled artisans meticulously craft every piece of ceramic to create beautiful and functional products that stand the test of time. With a global presence in over 50 countries, White Horse Ceramics is synonymous with style, durability, and unmatched craftsmanship. They offer a variety of tiles, including ceramic wall, floor, and porcelain tiles, as well as stone, mosaic, and quartz tiles. The company 

In [11]:
df_all_data.head()

,X,y
0,White Horse Tile Manufacturing | European Aest...,327
1,Wealth Solution Partners Super and SMSF Servic...,523
2,PMG Fire and Water Cleanup Services | Mold Rem...,562
3,TMP Capital PLLC Licensed in AL & FL | 203K Lo...,522
4,Genertek Power Industrial and Commercial Energ...,221


In [12]:
import torch

In [13]:
torch.cuda.is_available()

True

In [14]:
df_all_data.index

RangeIndex(start=0, stop=2000, step=1)

In [15]:
df_all_data['X'].tolist()

['White Horse Tile Manufacturing | European Aesthetics Ceramic Tiles | Ceramic Mosaics | Glossy Tile | Green-certified Tiles | Clearance Items | Ceramic Tiles Manufacturer | Quartz Surfaces White Horse is highly regarded as a tile trailblazer, designing ceramic tiles of the highest quality inspired by European aesthetics. White Horse Ceramic Singapore is a leading manufacturer of high-quality ceramics, offering a wide range of ceramic products for both residential and commercial spaces. Established in 1998, the company is known for its commitment to quality, innovation, and artistry. Their skilled artisans meticulously craft every piece of ceramic to create beautiful and functional products that stand the test of time. With a global presence in over 50 countries, White Horse Ceramics is synonymous with style, durability, and unmatched craftsmanship. They offer a variety of tiles, including ceramic wall, floor, and porcelain tiles, as well as stone, mosaic, and quartz tiles. The company

In [16]:
# Check if CUDA is available and set the device accordingly
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [17]:
device

device(type='cpu')

In [18]:
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)


def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['X'])
    tokenized_inputs['labels'] = examples['y']
    return tokenized_inputs


# define custom batch preprocessor
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d


# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

# create custom trainer class to be able to pass label weights and calculate mutilabel loss
class CustomTrainer(Trainer):
    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights.to(device)  # Move to device

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss

# set random seed
random.seed(0)

# load data
with open('../data/first_2k_fixed.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

# shuffle data
random.shuffle(data)

# reshape

idx = list(df_all_data.index)
text = df_all_data['X'].tolist()
labels = df_all_data['y'].to_numpy().reshape(-1,1)

# idx, text, labels = list(zip(*[(int(row.index), f'Title: {row[1].strip()}\n\nAbstract: {row[2].strip()}', row[3:]) for row in data]))
# labels = np.array(labels, dtype=int)

# create label weights
label_weights = 1 - labels.sum(axis=0) / labels.sum()

# stratified train test split for multilabel ds
row_ids = np.arange(len(labels))
train_idx, y_train, val_idx, y_val = iterative_train_test_split(row_ids[:,np.newaxis], labels, test_size = 0.1)
x_train = [text[i] for i in train_idx.flatten()]
x_val = [text[i] for i in val_idx.flatten()]

/root/anaconda3/envs/dipmaind/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [20]:
x_train

['White Horse Tile Manufacturing | European Aesthetics Ceramic Tiles | Ceramic Mosaics | Glossy Tile | Green-certified Tiles | Clearance Items | Ceramic Tiles Manufacturer | Quartz Surfaces White Horse is highly regarded as a tile trailblazer, designing ceramic tiles of the highest quality inspired by European aesthetics. White Horse Ceramic Singapore is a leading manufacturer of high-quality ceramics, offering a wide range of ceramic products for both residential and commercial spaces. Established in 1998, the company is known for its commitment to quality, innovation, and artistry. Their skilled artisans meticulously craft every piece of ceramic to create beautiful and functional products that stand the test of time. With a global presence in over 50 countries, White Horse Ceramics is synonymous with style, durability, and unmatched craftsmanship. They offer a variety of tiles, including ceramic wall, floor, and porcelain tiles, as well as stone, mosaic, and quartz tiles. The company

In [21]:
y_train

array([[327],
       [562],
       [333],
       ...,
       [441],
       [813],
       [523]])

In [22]:
x_val

["Wealth Solution Partners Super and SMSF Services | Financial Planning and Investment Management Services | Farm Succession Services | Financial Services | Life Stages Services | Construction Planning Services | Tax and Redundancy Services | Elderly Care Services | Risk Insurance Services WSP, Wealth Solution Partners, Financial Planning, Financial Planners, ATO, Xplan, Share Trading, Kyabram, advisor, term deposit, retirement, finances, estate, economical, pension, capital, wills, death benefits, insurance, investing, Lonsec, M&S, credit, broker, money, tax, taxation, cashflow, strategic, share trading Wealth Solution Partners Pty Ltd is an independent financial planning company that provides individual financial advice and investment strategies to help clients build a better lifestyle and future. They offer services such as retirement planning, estate planning, investment advice, risk management, and business planning. The company is not owned by a bank or other financial institutio

In [24]:
# create hf dataset
ds = DatasetDict({
    'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
    'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
})
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1800
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 200
    })
})

In [19]:
# model name
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

# load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=labels.shape[1]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id
model = model.to(device)

# define training args
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 16, # tested with 16gb gpu ram
    per_device_eval_batch_size = 16,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['val'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    label_weights = torch.tensor(label_weights, device=model.device)
)

# Assuming 'model' is your model and 'data' is a tensor or batch from your dataset
print(model.device)  # Check device of the model
# print(data.device)   # Check device of the data

trainer.train()

# save model
peft_model_id = 'multilabel_mistral'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

# load model
peft_model_id = 'multilabel_mistral'
model = AutoModelForSequenceClassification.from_pretrained(peft_model_id)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!ls

In [ ]:
df_all_data['y'].to_numpy().reshape(-1,1).shape